In [60]:
module Constraints

import Base: getindex, 
setindex!, 
convert,
==, <, <=, >, >=, +, -, *, /,
size, length 

import DataStructures: OrderedDict

abstract Sealed

type SealedScalar{T} <: Sealed
    data::T
    touched::Bool
end

for op in [==, <, <=, >, >=, +, -, *, /]
    op(s::SealedScalar, x::Number) = (s.touched = true; op(s.data, x))
    op(x::Number, s::SealedScalar) = (s.touched = true; op(s.data, x))
end

type SealedArray{T,N} <: Sealed
    data::Array{T,N}
    touched::Array{Bool,N}
end

function SealedArray{T,N}(data::Array{T,N})
    touched = falses(size(data)...)
    SealedArray{T,N}(data, touched)
end

function getindex{T,N}(s::SealedArray{T,N}, key...)
    s.touched[key...] = true
    s.data[key...]
end

function setindex!{T,N}(s::SealedArray{T,N}, value, key...)
    s.data[key...] = value
end

function seal!(s::SealedArray)
    s.touched[:] = false
end

for op in [size, length]
    op(s::SealedArray) = op(s.data)
end

type Constraint
    vars::Vector{Symbol}
    func::Function
end

immutable Domain
    lower::Array{Int}
    upper::Array{Int}
end

immutable Variable
    name::Symbol
    domain::Domain
end

length(v::Variable) = length(v.domain.lower)

type Problem
    vars::Vector{Variable}
    constraints::Vector{Constraint}
end

function Problem()
    Problem(Variable[], Constraint[])
end

function addVariable!(p::Problem, name::Symbol, lower::Array{Int}, upper::Array{Int})
    @assert length(upper) == length(lower)
    upper = reshape(upper, size(lower))
    push!(p.vars, Variable(name, Domain(lower, upper)))
end
 
function addConstraint!(p::Problem, vars::Vector{Symbol}, func::Function)
    push!(p.constraints, Constraint(vars, func))
end
    
typealias PotentialSolution OrderedDict{Symbol, SealedArray}
typealias Solution Dict{Symbol, Array}

function collect_domains(vars::Vector{Variable})
    total_vars = sum([length(v) for v in vars])
    min = Array(Int, total_vars)
    max = Array(Int, total_vars)
    offset = 0
    for v in vars
        numvar = length(v)
        min[offset+(1:numvar)] = v.domain.lower[:]
        max[offset+(1:numvar)] = v.domain.upper[:]
        offset += numvar
    end
    min, max
end

function min_solution(vars::Vector{Variable})
    PotentialSolution(zip([v.name for v in vars],
    [SealedArray(copy(v.domain.lower)) for v in vars]))
end
    
function increment!(potential_solution::PotentialSolution, increment_index::Integer, min::Vector{Int}, max::Vector{Int})
     for i = 1:increment_index-1
        potential_solution[i] = min[i]
    end
    potential_solution[increment_index] += 1
    for i = increment_index:length(min)-1
        if potential_solution[i] > max[i]
            potential_solution[i] = 0
            potential_solution[i+1] += 1
        end
    end
end

function solve(prob::Problem, max_solutions=Inf)
    min, max = collect_domains(prob.vars)
    @assert length(min) == length(max)
    numvar = length(min)
    touched = Array(Bool, numvar)
    potential_solution = min_solution(prob.vars)
    solutions = Solution[]
    num_satisfied_constraints = 0
    finished = false
    increment_index = 0
    num_nodes_explored = 1
    
    while length(solutions) < max_solutions && !finished
        for constraint in prob.constraints
#             @show potential_solution[:x].data
            seal!(potential_solution)
            if constraint.func([potential_solution[v] for v in constraint.vars]...)
                num_satisfied_constraints += 1
                if num_satisfied_constraints == length(prob.constraints)
                    push!(solutions, extract_solution(potential_solution))
                    increment_index = 1
                else
                    increment_index = 0
                end
            else
                num_satisfied_constraints = 0
                check_seals!(touched, potential_solution)
                increment_index = findfirst(touched)
            end
            if increment_index > 0
                increment!(potential_solution, increment_index, min, max)
                if potential_solution[numvar] > max[numvar]
                    finished = true
                    break
                end
                num_nodes_explored += 1
            end
        end
    end
    @show num_nodes_explored
    solutions
end
    
seal!(potential::PotentialSolution) = map(seal!, values(potential))


# function addVariable!(prob::Problem, name::Symbol, lower, upper)
    
    
    
function check_seals!(touched::Vector{Bool}, potential::PotentialSolution)
    offset = 0
    for (k, v) in potential
        numel = length(v.touched)
        for j = 1:numel
            touched[offset+j] = v.touched[j]
        end
        offset += numel
    end
end

function extract_solution(solution::OrderedDict{Symbol, SealedArray})
    result = Dict(zip(keys(solution), map(v -> copy(v.data), values(solution))))
end
        
function getindex(solution::PotentialSolution, index::Number)
    offset = 0
    for (k, v) in solution
        numel = length(v)
        if index <= offset + numel
            return v[index - offset]
        end
        offset += numel
    end
end

function setindex!(solution::OrderedDict{Symbol, SealedArray}, value, index::Number)
    offset = 0
    for (k, v) in solution
        numel = length(v)
        if index <= offset + numel
            v[index - offset] = value
            break
        end
        offset += numel
    end
end


# type SealedDict{T,N}

# type Seal{T}
#     data::T
#     touched
# end

# num_vars(s::Seal) = num

# function Seal{T}(data::T)
#     Seal(data, false)
# end

# function getindex(s::Seal, key...)
#     s.touched = true
#     getindex(s.data, key...)
# end
    
end

import Constraints

In [63]:
p = Constraints.Problem()
Constraints.addVariable!(p, :x, [0, 0], [10, 10])
Constraints.addConstraint!(p, [:x], x -> x[1] == x[2])
Constraints.addConstraint!(p, [:x], x -> x[1] == 5)
Constraints.addConstraint!(p, [:x], x -> x[2] == 5)

3-element Array{Constraints.Constraint,1}:
 Constraints.Constraint([:x],(anonymous function))
 Constraints.Constraint([:x],(anonymous function))
 Constraints.Constraint([:x],(anonymous function))

In [64]:
@time solutions = Constraints.solve(p)

num_nodes_explored = 38

1-element Array{Dict{Symbol,Array{T,N}},1}:
 Dict{Symbol,Array{T,N}}(:x=>[5,5])


  0.004737 seconds (1.07 k allocations: 49.873 KB)


In [ ]:
y = SealedArray(reshape(Float64[0.0], ()))

In [100]:
y[] = 1
y[]
seal!(y)
@show y
y[]
@show y

y = SealedArray{Float64,0}(1.0,false)

SealedArray{Float64,0}(0-dimensional Array{Float64,0}:
1.0,0-dimensional Array{Bool,0}:
true)


y = SealedArray{Float64,0}(1.0,true)


In [101]:
y[]

1.0

In [102]:
y == 1

false

In [83]:
d = Dict(:foo => 1, :bar => 2)

Dict{Symbol,Int64} with 2 entries:
  :bar => 2
  :foo => 1

In [84]:
getindex(d, :bar)

2

In [125]:
a = [1 2 3; 4 5 6]

2x3 Array{Int64,2}:
 1  2  3
 4  5  6

In [126]:
length(a)

6

In [127]:
size(a)

(2,3)

In [132]:
a[6]

6

In [117]:
z = SealedScalar(1.0, false)

SealedScalar{Float64}(1.0,false)

In [113]:
z == 1

true

In [118]:
z < 2

true

In [119]:
z

SealedScalar{Float64}(1.0,true)

In [95]:
if z == 1
    @show z
end

true

In [135]:
q = 1
q[1]
length(q)

1

In [144]:
?findfirst

search: 

```
findfirst(A,v)
```

Return the index of the first element equal to `v` in `A`.

```
findfirst(A)
```

Return the index of the first non-zero value in `A` (determined by `A[i]!=0`).

```
findfirst(predicate, A)
```

Return the index of the first element of `A` for which `predicate` returns `true`.


findfirst

